In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_test = pd.read_csv('C:/Users/Hrishikesh/Desktop/Python/Leetcode Problems/Machine_learning/test_leads.csv')
df_train = pd.read_csv('C:/Users/Hrishikesh/Desktop/Python/Leetcode Problems/Machine_learning/train_leads.csv')

In [ ]:
sns.heatmap(df_train.corr(), cmap='YlGnBu')
plt.show()

#Check correlations between different fields:
#Seems like User Activity : 2,4,5,9,10,11 have a greater positive impact on the purchase decision
#While User Activity 1,6,7 and 8 have fairly negative correlation 

In [ ]:
df_train.info() #Check for Null values

In [ ]:
#80-20 training and testing on the training data provided : Stratified shuffle split
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
for train_indices, test_indices in split.split(df_train, df_train[['buy','user_activity_var_2','user_activity_var_4','user_activity_var_9']]):
    strat_train_set = df_train.loc[train_indices]
    strat_test_set = df_train.loc[test_indices]


In [ ]:
plt.subplot(1,2,1)
strat_train_set['buy'].hist()
strat_train_set['user_activity_var_9'].hist()

plt.subplot(1,2,2)
strat_test_set['buy'].hist()
strat_test_set['user_activity_var_9'].hist()

plt.show()

In [ ]:
#checking null values
strat_train_set.info()
strat_test_set.info()

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
class FeatureDropper(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.drop(['created_at', 'campaign_var_1','campaign_var_2', 'products_purchased','signup_date','user_activity_var_1','user_activity_var_3', 'user_activity_var_6','user_activity_var_7','user_activity_var_8','user_activity_var_10','user_activity_var_12'], axis=1, errors="ignore")

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('featuredropper', FeatureDropper())])

In [ ]:
strat_train_set = pipeline.fit_transform(strat_train_set)

In [ ]:
strat_train_set.info()

In [ ]:
from sklearn.preprocessing import StandardScaler

X = strat_train_set.drop(['buy'],axis = 1)
y = strat_train_set['buy']

scaler = StandardScaler()
X_data = scaler.fit_transform(X)
y_data = y.to_numpy()

In [ ]:
X_data

In [ ]:
#using random forest classifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
clf = RandomForestClassifier()

param_grid = [
    {'n_estimators': [10,100,200,500,1000], 'max_depth': [None, 5, 10], 'min_samples_split':[2,3,4]}
]

grid_search = GridSearchCV(clf, param_grid, cv=3, scoring ='accuracy', return_train_score = True)
grid_search.fit(X_data, y_data)

In [ ]:
final_clf = grid_search.best_estimator_

In [ ]:
final_clf

In [ ]:
strat_test_set = pipeline.fit_transform(strat_test_set)

In [ ]:
strat_test_set

In [ ]:
X_test = strat_test_set.drop(['buy'],axis=1)
y_test = strat_test_set['buy']

scaler = StandardScaler()
X_data_test = scaler.fit_transform(X_test)
y_data_test = y_test.to_numpy()

In [ ]:
final_clf.score(X_data_test, y_data_test)

In [ ]:
final_leads_data = pipeline.fit_transform(df_train)

In [ ]:
final_leads_data

In [ ]:
X_final = final_leads_data.drop(['buy'],axis=1)
y_final = final_leads_data['buy']

scaler = StandardScaler()
X_leads_final = scaler.fit_transform(X_final)
y_leads_final = y_final.to_numpy()

In [ ]:
prod_clf = RandomForestClassifier()

param_grid = [
    {'n_estimators': [10,100,200,500,1000], 'max_depth': [None, 5, 10], 'min_samples_split':[2,3,4]}
]

grid_search = GridSearchCV(prod_clf, param_grid, cv=3, scoring ='accuracy', return_train_score = True)
grid_search.fit(X_leads_final, y_leads_final)

In [ ]:
prod_final_clf = grid_search.best_estimator_

In [ ]:
prod_final_clf

In [ ]:
final_test_data = pipeline.fit_transform(df_test)

In [ ]:
final_test_data.info()

In [ ]:
X_final_data = final_test_data

scaler = StandardScaler()
X_leads_final_test = scaler.fit_transform(X_final_data)

In [ ]:
predictions = prod_final_clf.predict(X_leads_final_test)

In [ ]:
sum(predictions) #to check if there are both 1's and 0's

In [ ]:
final_data_frame = pd.DataFrame(df_test['id'])
final_data_frame['buy'] = predictions

final_data_frame.to_csv('C:/Users/Hrishikesh/Desktop/Python/Leetcode Problems/Machine_learning/submission_v1.csv',index=False)